# FEMA Disaster Declarations Summaries
Author: Mark Bauer

In [1]:
# import packages
import duckdb
from datetime import datetime

In [2]:
# reproducibility
%reload_ext watermark
%watermark -v -p duckdb

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.6.0

duckdb: 1.0.0



In [3]:
# data retrieved
current_date = datetime.now()
print(f"The data was retrieved on {current_date.strftime('%Y-%m-%d')}.")

The data was retrieved on 2024-10-06.


# OpenFEMA Dataset: Disaster Declarations Summaries - v2

## Dataset
Federal Emergency Management Agency (FEMA), OpenFEMA Dataset: Disaster Declarations Summaries - v2. Retrieved from https://www.fema.gov/openfema-data-page/disaster-declarations-summaries-v2. This product uses the FEMA OpenFEMA API, but is not endorsed by FEMA. The Federal Government or FEMA cannot vouch for the data or analyses derived from these data after the data have been retrieved from the Agency's website(s).

Read more about [OpenFEMA Terms and Conditions](https://www.fema.gov/about/openfema/terms-conditions).

## Dataset Description
>Disaster Declarations Summaries is a summarized dataset describing all federally declared disasters. This dataset lists all official FEMA Disaster Declarations, beginning with the first disaster declaration in 1953 and features all three disaster declaration types: major disaster, emergency, and fire management assistance. The dataset includes declared recovery programs and geographic areas (county not available before 1964; Fire Management records are considered partial due to historical nature of the dataset).

Source: [OpenFEMA Dataset: Disaster Declarations Summaries - v2](https://www.fema.gov/openfema-data-page/disaster-declarations-summaries-v2)

## Disaster Delcarations
For more information on the disaster declaration process:
- Information about disasters: https://www.fema.gov/disasters
- How a disaster is declared: https://www.fema.gov/disasters/how-declared

# Additional Resources
To examine other cool data visualizations about this data, visit FEMA's [Disaster Declarations for States and Counties](https://www.fema.gov/data-visualization/disaster-declarations-states-and-counties) page.

# Read In Data

In [4]:
# duckdb connection
con = duckdb.connect()

# create disaster_declarations table
con.execute("""
    CREATE TABLE disaster_declarations
    AS FROM read_parquet('https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries.parquet')
"""
)

# sanity check
sql = """
    SELECT *
    FROM disaster_declarations
    LIMIT 10
"""

con.sql(sql)

┌──────────────────────┬────────────────┬─────────┬───┬──────────────────────┬──────────────────────┐
│          id          │ disasterNumber │  state  │ … │     lastRefresh      │         hash         │
│       varchar        │     int16      │ varchar │   │      timestamp       │       varchar        │
├──────────────────────┼────────────────┼─────────┼───┼──────────────────────┼──────────────────────┤
│ f15a7a79-f1c3-41bb…  │           5530 │ NV      │ … │ 2024-08-27 18:22:1…  │ 5d07e7c51bb300bfbe…  │
│ 09e3f81a-5e16-4b72…  │           5529 │ OR      │ … │ 2024-08-27 18:22:1…  │ ae87cf3c6ed795015b…  │
│ 59983f89-30bf-4888…  │           5528 │ OR      │ … │ 2024-08-27 18:22:1…  │ 432cf0995c47e3895c…  │
│ 8d13ecf0-bc2f-496b…  │           5527 │ OR      │ … │ 2024-08-27 18:22:1…  │ 2f21d90cb6bc64b0d4…  │
│ 17c24d4a-49a9-4cac…  │           5526 │ CO      │ … │ 2024-08-27 18:22:1…  │ e753ba692156f389db…  │
│ f1140a27-cb85-404c…  │           5525 │ CO      │ … │ 2024-08-27 18:22:1…  │ b1f

# Describe and Summarize Data

In [5]:
# count rows
con.sql("SELECT COUNT(id) AS count_rows FROM disaster_declarations")

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│      66978 │
└────────────┘

In [6]:
# examine column datatypes
sql = "DESCRIBE SELECT * FROM disaster_declarations"
describe_relation = con.sql(sql)

# select desired columns
(con
 .sql("SELECT column_name, column_type FROM describe_relation")
 .show(max_rows=50)
)

┌──────────────────────────┬─────────────┐
│       column_name        │ column_type │
│         varchar          │   varchar   │
├──────────────────────────┼─────────────┤
│ id                       │ VARCHAR     │
│ disasterNumber           │ SMALLINT    │
│ state                    │ VARCHAR     │
│ femaDeclarationString    │ VARCHAR     │
│ declarationType          │ VARCHAR     │
│ declarationDate          │ DATE        │
│ fyDeclared               │ SMALLINT    │
│ incidentType             │ VARCHAR     │
│ declarationTitle         │ VARCHAR     │
│ ihProgramDeclared        │ BOOLEAN     │
│ iaProgramDeclared        │ BOOLEAN     │
│ paProgramDeclared        │ BOOLEAN     │
│ hmProgramDeclared        │ BOOLEAN     │
│ incidentBeginDate        │ DATE        │
│ incidentEndDate          │ DATE        │
│ disasterCloseoutDate     │ DATE        │
│ tribalRequest            │ BOOLEAN     │
│ fipsStateCode            │ VARCHAR     │
│ fipsCountyCode           │ VARCHAR     │
│ placeCode

In [7]:
# summary statistics of columns
sql = "SUMMARIZE SELECT * FROM disaster_declarations"
summarize_relation = con.sql(sql)

# fetch rows as a pandas df for readability
summarize_relation.df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,id,VARCHAR,00002fed-4c4b-49c0-8031-6e9749819fde,ffffdc28-ff85-43c7-91bb-f40fc4932471,65550,None,None,None,None,None,66978,0.00
1,disasterNumber,SMALLINT,1,5538,5081,2655.3295858341544,1428.7185809470966,1358,3132,3712,66978,0.00
2,state,VARCHAR,AK,WY,59,None,None,None,None,None,66978,0.00
3,femaDeclarationString,VARCHAR,DR-1-GA,FM-5538-CA,5020,None,None,None,None,None,66978,0.00
4,declarationType,VARCHAR,DR,FM,3,None,None,None,None,None,66978,0.00
5,declarationDate,DATE,1953-05-02,2024-10-04,3516,None,None,None,None,None,66978,0.00
6,fyDeclared,SMALLINT,1953,2025,74,2005.1754755292782,14.989837665265801,1997,2008,2019,66978,0.00
7,incidentType,VARCHAR,Biological,Winter Storm,26,None,None,None,None,None,66978,0.00
8,declarationTitle,VARCHAR,HURRICANE EARL,ZOGG FIRE,2349,None,None,None,None,None,66978,0.00
9,ihProgramDeclared,BOOLEAN,false,true,2,None,None,None,None,None,66978,0.00


In [8]:
# examine NULL percentage
sql = """
    SELECT column_name, column_type, null_percentage
    FROM summarize_relation
    ORDER BY null_percentage DESC
"""

con.sql(sql).show(max_rows=50)

┌──────────────────────────┬─────────────┬─────────────────┐
│       column_name        │ column_type │ null_percentage │
│         varchar          │   varchar   │  decimal(9,2)   │
├──────────────────────────┼─────────────┼─────────────────┤
│ lastIAFilingDate         │ DATE        │           71.89 │
│ designatedIncidentTypes  │ VARCHAR     │           71.38 │
│ disasterCloseoutDate     │ DATE        │           23.54 │
│ incidentEndDate          │ DATE        │            1.70 │
│ id                       │ VARCHAR     │            0.00 │
│ disasterNumber           │ SMALLINT    │            0.00 │
│ state                    │ VARCHAR     │            0.00 │
│ femaDeclarationString    │ VARCHAR     │            0.00 │
│ declarationType          │ VARCHAR     │            0.00 │
│ declarationDate          │ DATE        │            0.00 │
│ fyDeclared               │ SMALLINT    │            0.00 │
│ incidentType             │ VARCHAR     │            0.00 │
│ declarationTitle      

# Preview Values
Scan table twice to preview all columns.

In [9]:
sql = """
    SELECT
        id,
        disasterNumber,
        state,
        femaDeclarationString,
        declarationType,
        declarationDate,
        fyDeclared,
        incidentType,
        declarationTitle,
        ihProgramDeclared,
        iaProgramDeclared,
        paProgramDeclared,
        hmProgramDeclared
    FROM
        disaster_declarations
    LIMIT 5   
"""

con.sql(sql).df()

,id,disasterNumber,state,femaDeclarationString,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared
0,f15a7a79-f1c3-41bb-8a5c-c05fbae34423,5530,NV,FM-5530-NV,FM,2024-08-12,2024,Fire,GOLD RANCH FIRE,False,False,True,True
1,09e3f81a-5e16-4b72-b317-1c64e0cfa59c,5529,OR,FM-5529-OR,FM,2024-08-09,2024,Fire,LEE FALLS FIRE,False,False,True,True
2,59983f89-30bf-4888-b21b-62e8d57d9aac,5528,OR,FM-5528-OR,FM,2024-08-06,2024,Fire,ELK LANE FIRE,False,False,True,True
3,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0,5527,OR,FM-5527-OR,FM,2024-08-02,2024,Fire,MILE MARKER 132 FIRE,False,False,True,True
4,17c24d4a-49a9-4cac-9322-e5427c4cdfeb,5526,CO,FM-5526-CO,FM,2024-08-01,2024,Fire,QUARRY FIRE,False,False,True,True


In [10]:
sql = """
    SELECT
        incidentBeginDate,
        incidentEndDate,
        disasterCloseoutDate,
        tribalRequest,
        fipsStateCode,
        fipsCountyCode,
        placeCode,
        designatedArea,
        declarationRequestNumber,
        lastIAFilingDate,
        incidentId,
        region,
        designatedIncidentTypes,
        lastRefresh,
        hash
    FROM
        disaster_declarations
    LIMIT 5  
"""

con.sql(sql).df()

,incidentBeginDate,incidentEndDate,disasterCloseoutDate,tribalRequest,fipsStateCode,fipsCountyCode,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,incidentId,region,designatedIncidentTypes,lastRefresh,hash
0,2024-08-11,NaT,NaT,False,32,031,99031,Washoe (County),24123,NaT,2024081201,9,R,2024-08-27 18:22:14.800,5d07e7c51bb300bfbec94a699a1e1ab1d61a97cd
1,2024-08-08,NaT,NaT,False,41,067,99067,Washington (County),24122,NaT,2024081001,10,R,2024-08-27 18:22:14.800,ae87cf3c6ed795015b714af7166c7c295b2b67c7
2,2024-08-04,NaT,NaT,False,41,031,99031,Jefferson (County),24116,NaT,2024080701,10,R,2024-08-27 18:22:14.800,432cf0995c47e3895cea696ede5621b810460501
3,2024-08-02,NaT,NaT,False,41,017,99017,Deschutes (County),24111,NaT,2024080301,10,R,2024-08-27 18:22:14.800,2f21d90cb6bc64b0d4121aa3f18d852bbb4b11fa
4,2024-07-30,NaT,NaT,False,08,059,99059,Jefferson (County),24106,NaT,2024080102,8,R,2024-08-27 18:22:14.800,e753ba692156f389dbe19f7a1c332d04ae145f74


In [11]:
# earliest event
sql = """
    SELECT
        disasterNumber,
        state,
        femaDeclarationString,
        declarationType,
        declarationDate,
        fyDeclared,
        incidentType,
        declarationTitle
    FROM disaster_declarations
    ORDER BY declarationDate ASC
    LIMIT 1
"""

con.sql(sql).df()

,disasterNumber,state,femaDeclarationString,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle
0,1,GA,DR-1-GA,DR,1953-05-02,1953,Tornado,TORNADO


In [12]:
# latest event
sql = """
    SELECT
        disasterNumber,
        state,
        femaDeclarationString,
        declarationType,
        declarationDate,
        fyDeclared,
        incidentType,
        declarationTitle
    FROM disaster_declarations
    ORDER BY declarationDate DESC
    LIMIT 1
"""

con.sql(sql).df()

,disasterNumber,state,femaDeclarationString,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle
0,4833,AZ,DR-4833-AZ,DR,2024-10-04,2025,Fire,WATCH FIRE


In [13]:
# event last refreshed
sql = """
    SELECT
        lastRefresh,
        disasterNumber,
        state,
        femaDeclarationString,
        declarationType,
        declarationDate,
        fyDeclared,
        incidentType,
        declarationTitle
    FROM disaster_declarations
    ORDER BY lastRefresh DESC
    LIMIT 1
"""

con.sql(sql).df()

,lastRefresh,disasterNumber,state,femaDeclarationString,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle
0,2024-10-05 23:02:06.173,4829,SC,DR-4829-SC,DR,2024-09-29,2024,Hurricane,HURRICANE HELENE


In [14]:
# count per declaration
sql = """
    SELECT
       femaDeclarationString,
       count(id) AS count
    FROM
        disaster_declarations
    GROUP BY femaDeclarationString
    ORDER BY count DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY count DESC
    LIMIT 20
""")

┌───────────────────────┬───────┐
│ femaDeclarationString │ count │
│        varchar        │ int64 │
├───────────────────────┼───────┤
│ DR-4522-ME            │   443 │
│ DR-4485-TX            │   257 │
│ EM-3458-TX            │   257 │
│ DR-1239-TX            │   257 │
│ EM-3261-TX            │   255 │
│ DR-4586-TX            │   254 │
│ DR-1606-TX            │   254 │
│ EM-3554-TX            │   254 │
│ EM-3216-TX            │   254 │
│ DR-1624-TX            │   254 │
│ EM-3284-TX            │   229 │
│ EM-3142-TX            │   227 │
│ DR-4527-SD            │   207 │
│ EM-3444-ME            │   179 │
│ DR-4501-GA            │   160 │
│ EM-3464-GA            │   160 │
│ EM-3616-GA            │   159 │
│ EM-3218-GA            │   159 │
│ EM-3387-GA            │   159 │
│ DR-4338-GA            │   159 │
├───────────────────────┴───────┤
│ 20 rows             2 columns │
└───────────────────────────────┘

In [15]:
# examine count unique for femaDeclarationString = 'DR-4522-ME'
(con
 .sql("""
    SELECT *
    FROM disaster_declarations
    WHERE femaDeclarationString = 'DR-4522-ME'
""")
 .df()
 .describe(include='object')
 .loc['unique']
)

id                          443
state                         1
femaDeclarationString         1
declarationType               1
incidentType                  1
declarationTitle              1
fipsStateCode                 1
fipsCountyCode               17
placeCode                   443
designatedArea              438
declarationRequestNumber      1
incidentId                    1
designatedIncidentTypes       0
hash                        443
Name: unique, dtype: object

The duplicate records for the same declaration suggest this dataset is grouped by **declaration** and **places** (i.e. placeCode) that received the declaration. If we're only interested in declarations, we need to return unique FEMA declarations.

**Table xx.** Count of Disasters Declared

In [16]:
# drop duplicate FEMA declarations
con.sql("""
    SELECT COUNT(DISTINCT femaDeclarationString) AS disasters_declared
    FROM disaster_declarations  
""")

┌────────────────────┐
│ disasters_declared │
│       int64        │
├────────────────────┤
│               4984 │
└────────────────────┘

# Analysis

**Table xx.** Count of Disasters Declared by Declaration Type

In [17]:
sql = """
    WITH declarations AS (
    
        SELECT
            DISTINCT ON (femaDeclarationString) femaDeclarationString,
            declarationType,
            id           
        FROM
            disaster_declarations
    )
    
    SELECT
       declarationType,
       count(id) AS count
    FROM
        declarations
    GROUP BY declarationType
    ORDER BY count DESC   
"""

con.sql(sql)

┌─────────────────┬───────┐
│ declarationType │ count │
│     varchar     │ int64 │
├─────────────────┼───────┤
│ DR              │  2826 │
│ FM              │  1537 │
│ EM              │   621 │
└─────────────────┴───────┘

**Table xx.** Top 20 Years with the Highest Number of Disaster Declarations

In [18]:
sql = """
    WITH declarations AS (
    
        SELECT
            DISTINCT ON (femaDeclarationString) femaDeclarationString,
            fyDeclared,
            id           
        FROM
            disaster_declarations
    )
    
    SELECT
        fyDeclared AS year,
        count(id) AS count
    FROM
        declarations
    GROUP BY fyDeclared
    ORDER BY count DESC
"""

relation = con.sql(sql)

con.sql("""
    SELECT row_number() OVER () AS rank, *
    FROM relation
    ORDER BY count DESC
    LIMIT 20
""")

┌───────┬───────┬───────┐
│ rank  │ year  │ count │
│ int64 │ int16 │ int64 │
├───────┼───────┼───────┤
│     1 │  2020 │   315 │
│     2 │  2011 │   236 │
│     3 │  2024 │   160 │
│     4 │  2006 │   151 │
│     5 │  1996 │   149 │
│     6 │  2008 │   141 │
│     7 │  2005 │   139 │
│     8 │  2017 │   135 │
│     9 │  2007 │   133 │
│    10 │  2018 │   132 │
│    11 │  2004 │   129 │
│    12 │  2002 │   125 │
│    13 │  2009 │   125 │
│    14 │  1998 │   124 │
│    15 │  2021 │   123 │
│    16 │  2003 │   115 │
│    17 │  2023 │   114 │
│    18 │  2013 │   111 │
│    19 │  2010 │   109 │
│    20 │  1999 │   106 │
├───────┴───────┴───────┤
│ 20 rows     3 columns │
└───────────────────────┘

**Table xx.** Top 20 States with the Highest Number of Disaster Declarations

In [19]:
sql = """
    WITH declarations AS (
    
        SELECT
            DISTINCT ON (femaDeclarationString) femaDeclarationString,
            state,
            id           
        FROM
            disaster_declarations
    )
    
    SELECT
       state,
       count(id) AS count
    FROM
        declarations
    GROUP BY state
    ORDER BY count DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT row_number() OVER () AS rank, *
    FROM relation
    ORDER BY count DESC
    LIMIT 20
""")

┌───────┬─────────┬───────┐
│ rank  │  state  │ count │
│ int64 │ varchar │ int64 │
├───────┼─────────┼───────┤
│     1 │ CA      │   384 │
│     2 │ TX      │   376 │
│     3 │ OK      │   230 │
│     4 │ WA      │   208 │
│     5 │ FL      │   182 │
│     6 │ OR      │   158 │
│     7 │ NM      │   118 │
│     8 │ AZ      │   116 │
│     9 │ NY      │   116 │
│    10 │ LA      │   108 │
│    11 │ NV      │   106 │
│    12 │ CO      │   105 │
│    13 │ MT      │   104 │
│    14 │ AL      │   102 │
│    15 │ MS      │    94 │
│    16 │ TN      │    93 │
│    17 │ SD      │    92 │
│    18 │ KY      │    90 │
│    19 │ KS      │    88 │
│    20 │ AK      │    83 │
├───────┴─────────┴───────┤
│ 20 rows       3 columns │
└─────────────────────────┘

**Table xx.** Number of Disaster Declarations by Incident Type

In [20]:
sql = """
    WITH declarations AS (
    
        SELECT
            DISTINCT ON (femaDeclarationString) femaDeclarationString,
            incidentType,
            id           
        FROM
            disaster_declarations
    )
    
    SELECT
       incidentType,
       count(id) AS count
    FROM
        declarations
    GROUP BY incidentType
    ORDER BY count DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT row_number() OVER () AS rank, *
    FROM relation
    ORDER BY count DESC
""").show(max_rows=50)

┌───────┬─────────────────────┬───────┐
│ rank  │    incidentType     │ count │
│ int64 │       varchar       │ int64 │
├───────┼─────────────────────┼───────┤
│     1 │ Fire                │  1636 │
│     2 │ Severe Storm        │  1093 │
│     3 │ Flood               │   898 │
│     4 │ Hurricane           │   449 │
│     5 │ Tornado             │   182 │
│     6 │ Snowstorm           │   171 │
│     7 │ Biological          │   167 │
│     8 │ Severe Ice Storm    │    74 │
│     9 │ Typhoon             │    58 │
│    10 │ Drought             │    46 │
│    11 │ Earthquake          │    36 │
│    12 │ Other               │    34 │
│    13 │ Coastal Storm       │    31 │
│    14 │ Tropical Storm      │    29 │
│    15 │ Freezing            │    18 │
│    16 │ Winter Storm        │    13 │
│    17 │ Mud/Landslide       │    12 │
│    18 │ Toxic Substances    │     8 │
│    19 │ Dam/Levee Break     │     6 │
│    20 │ Fishing Losses      │     6 │
│    21 │ Volcanic Eruption   │     6 │


**Table xx.** Top 20 States and Incident Types with the Highest Number of Disaster Declarations

In [21]:
sql = """
    WITH declarations AS (
    
        SELECT
            DISTINCT ON (femaDeclarationString) femaDeclarationString,
            state,
            incidentType,
            id           
        FROM
            disaster_declarations
    )
    
    SELECT
       state,
       incidentType,
       count(id) AS count
    FROM
        declarations
    GROUP BY
        state, incidentType
    ORDER BY
        count DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT row_number() OVER () AS rank, *
    FROM relation
    ORDER BY count DESC
    LIMIT 20
""")

┌───────┬─────────┬──────────────┬───────┐
│ rank  │  state  │ incidentType │ count │
│ int64 │ varchar │   varchar    │ int64 │
├───────┼─────────┼──────────────┼───────┤
│     1 │ CA      │ Fire         │   287 │
│     2 │ TX      │ Fire         │   260 │
│     3 │ WA      │ Fire         │   142 │
│     4 │ OR      │ Fire         │   115 │
│     5 │ OK      │ Fire         │   111 │
│     6 │ NV      │ Fire         │    80 │
│     7 │ AZ      │ Fire         │    80 │
│     8 │ CO      │ Fire         │    79 │
│     9 │ NM      │ Fire         │    71 │
│    10 │ FL      │ Fire         │    67 │
│    11 │ MT      │ Fire         │    66 │
│    12 │ FL      │ Hurricane    │    55 │
│    13 │ OK      │ Severe Storm │    50 │
│    14 │ AL      │ Severe Storm │    45 │
│    15 │ TN      │ Severe Storm │    43 │
│    16 │ KS      │ Severe Storm │    43 │
│    17 │ TX      │ Flood        │    41 │
│    18 │ CA      │ Flood        │    41 │
│    19 │ MS      │ Severe Storm │    41 │
│    20 │ M

**Table xx.** Number of Tribal Disaster Declarations

In [22]:
sql = """
    WITH declarations AS (
    
        SELECT
            DISTINCT ON (femaDeclarationString) femaDeclarationString,
            tribalRequest,
            id           
        FROM
            disaster_declarations
    )
    
    SELECT
       tribalRequest,
       count(id) AS count
    FROM
        declarations
    GROUP BY tribalRequest
    ORDER BY count DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY count DESC
""")

┌───────────────┬───────┐
│ tribalRequest │ count │
│    boolean    │ int64 │
├───────────────┼───────┤
│ false         │  4877 │
│ true          │   107 │
└───────────────┴───────┘

**Table xx.** Top 20 States and Incident Types with the Highest Number of Tribal Disaster Declarations

In [23]:
sql = """
    WITH declarations AS (
    
        SELECT
            DISTINCT ON (femaDeclarationString) femaDeclarationString,
            state,
            incidentType,
            id           
        FROM
            disaster_declarations
        WHERE
            tribalRequest = true
    )
    
    SELECT
       state,
       incidentType,
       count(id) AS count
    FROM
        declarations
    GROUP BY state, incidentType
    ORDER BY count DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT row_number() OVER () AS rank, *
    FROM relation
    ORDER BY count DESC
    LIMIT 20
""")

┌───────┬─────────┬──────────────┬───────┐
│ rank  │  state  │ incidentType │ count │
│ int64 │ varchar │   varchar    │ int64 │
├───────┼─────────┼──────────────┼───────┤
│     1 │ OK      │ Biological   │    15 │
│     2 │ NM      │ Biological   │    11 │
│     3 │ CA      │ Severe Storm │     9 │
│     4 │ AZ      │ Severe Storm │     4 │
│     5 │ FL      │ Hurricane    │     4 │
│     6 │ NE      │ Biological   │     4 │
│     7 │ OK      │ Severe Storm │     4 │
│     8 │ SD      │ Biological   │     3 │
│     9 │ KS      │ Biological   │     3 │
│    10 │ FL      │ Biological   │     2 │
│    11 │ WA      │ Fire         │     2 │
│    12 │ NY      │ Biological   │     2 │
│    13 │ MN      │ Biological   │     2 │
│    14 │ NM      │ Flood        │     2 │
│    15 │ SD      │ Severe Storm │     2 │
│    16 │ AL      │ Biological   │     2 │
│    17 │ AZ      │ Flood        │     2 │
│    18 │ SD      │ Winter Storm │     2 │
│    19 │ MA      │ Biological   │     2 │
│    20 │ N

**Table xx.** Top 20 States, Places, Designated Areas, and Incident Types with the Highest Number of Disaster Declarations

In [24]:
sql = """
    SELECT
        state,
        NULLIF(placeCode, 0) AS placeCode,
        designatedArea,
        incidentType,
        count(id) AS count
    FROM
        disaster_declarations
    GROUP BY ALL
    ORDER BY count DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT row_number() OVER () AS rank, * 
    FROM relation
    ORDER BY count DESC
    LIMIT 20
""")

┌───────┬─────────┬───────────┬───────────────────────────────┬──────────────┬───────┐
│ rank  │  state  │ placeCode │        designatedArea         │ incidentType │ count │
│ int64 │ varchar │  varchar  │            varchar            │   varchar    │ int64 │
├───────┼─────────┼───────────┼───────────────────────────────┼──────────────┼───────┤
│     1 │ TX      │ NULL      │ Statewide                     │ Fire         │    92 │
│     2 │ CA      │ 99037     │ Los Angeles (County)          │ Fire         │    55 │
│     3 │ CA      │ 99065     │ Riverside (County)            │ Fire         │    37 │
│     4 │ NV      │ 99031     │ Washoe (County)               │ Fire         │    34 │
│     5 │ CA      │ 99071     │ San Bernardino (County)       │ Fire         │    33 │
│     6 │ LA      │ 99075     │ Plaquemines (Parish)          │ Hurricane    │    32 │
│     7 │ WA      │ 99007     │ Chelan (County)               │ Fire         │    32 │
│     8 │ LA      │ 99057     │ Lafourche (